In [3]:
import rascaline
rascaline._c_lib._get_library()

from copy import deepcopy

import numpy as np

from multi_spectra_script import MultiSpectraModule
from equisolve.numpy.models.linear_model import Ridge
from equisolve.numpy.preprocessing import StandardScaler
from equisolve.utils.convert import ase_to_tensormap

import ase.io


In [4]:
frames = ase.io.read("water_converted.extxyz", ":20")

In [5]:
HYPERS = {
    "cutoff": 3.0,
    "max_radial": 6,
    "max_angular": 4,
    "atomic_gaussian_width": 0.3,
    "center_atom_weight": 1.0,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
}
multi_spectra_hypers = {}
#multi_spectra_hypers['Composition'] = {}
multi_spectra_hypers['SoapRadialSpectrum'] = deepcopy(HYPERS)
multi_spectra_hypers['SoapRadialSpectrum'].pop('max_angular')
multi_spectra_hypers['SoapPowerSpectrum'] = deepcopy(HYPERS)

In [6]:
y = ase_to_tensormap(frames, energy="TotEnergy", forces="force")

In [7]:
# Because all frames have the same species, we cannot scale, so we just subtract the mean
#transformer_composition = StandardScaler(parameter_keys=["values", "positions"], with_mean=True, with_std=False)
# THIS IS BUGGY, something wrong in the standardizer when setting these flags to False

transformer = StandardScaler(parameter_keys=["values", "positions"])
estimator = Ridge(parameter_keys=["values", "positions"])
gradients = ["positions"]
module = MultiSpectraModule(multi_spectra_hypers,
                            gradients, # gradients argument will
                            #transformer_X=transformer,
                            #transformer_y=transformer,
                            estimator=estimator)

# PR COMMENT have to think how to support both
# ase.Atoms as input and TensorMaps
Xi = module.compute(systems=frames)
module.fit(Xi, y, **{"estimator_kwargs": {"alpha": 1e-15}})
module.score(Xi, y)

671.9636307759743

In [6]:
# FOR DEBUGGING
#import importlib
#import equisolve.module
#importlib.reload(equisolve.module)

In [7]:
import equisolve.module
equisolve.module.save(module, "multi_spectra_script.py", "multi_spectra_script-water-dummy")

In [8]:
# RESTART NOTEBOOK

In [9]:
import equisolve.module
module = equisolve.module.load("multi_spectra_script-water-dummy")
print(module)

In [10]:
# in ipi
import ase.io
import numpy as np

structure = ase.io.read("h5o2+.extxyz", "0")

y_pred = module.forward(structure)
energy = y_pred.block().values[0][0]
forces = np.array(y_pred.block().gradient("positions").data.reshape(-1, 3))

print(energy)
print()
print(forces)

-29201.186821558375

[[-295.60671847   -4.62718863  -55.59864334]
 [ 298.54620907    4.04942317   37.15507667]
 [-221.68776379 -150.28232975 -242.81915489]
 [  -7.67013877    1.6028996    48.61326349]
 [ 176.03569875 -146.94124088  179.13481445]
 [ 286.69810363  136.65180697 -172.50009028]
 [-220.83316507  156.29491821  107.1942928 ]]


In [3]:
"""Interface with equiscript to run machine learning potentials"""
from equisolve.md import IpiDriver

Equisolve_driver = IpiDriver

Equisolve_driver("multi_spectra_script-water-dummy.zip,h5o2+.extxyz")